# 콜금리 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup

total = []
for n in range(1,501):                                         #URL은 날짜마다 range가 달라지므로 꼭 검색해보고 넣길
    url = "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page="+str(n)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tr_list = soup.select("tbody tr")                          #데이트와 종가가 나와있는 tr을 크롤링



    for tr in tr_list:                                        #for문을 통해 tr을 저장
        result={}
        result['date'] = tr.select_one("td.date").text.strip()      #result 딕셔너리에다가 데이트 삽입
    
        tdn = tr.select("td.num")
        result['callrate'] = tdn[0].text                      #td.num 을 추출하면 td.num의 리스트가 추출됨 . 그중에 인덱스 0인것만 뽑기
    
        total.append(result)


    
    


# 콜금리 데이터프레임화

In [15]:
import pandas as pd
from pandas import DataFrame as df
df1= []
df2= []

total2 = total[3:3473] #df1만들기
df1 = df(total2)


total3 = total[24:3494] #df2 만들기
df2 = df(total3)





newcols = {
    'date': '한달전 date', 
    'callrate': '한달전 callrate', 
}


df2.rename(columns=newcols, inplace=True)  #df1컬럼네임 바꾸기


df3=pd.concat([df1, df2], axis=1)          #두개의 df를  강제로 합치기 위해 concat을 사용. axis=1 옵션을 추가하여 옆에다가 합쳐짐

def get_label(df):                         # labelig 추가
    if df['callrate'] > df['한달전 callrate']:
        return "up"
    elif df['callrate'] < df['한달전 callrate']:
        return "down"
    else:
        return "middle"
df3['labeling']=df3.apply(get_label,axis=1) #axis=1 옵션을 넣어 적용하기

df3



,date,callrate,한달전 date,한달전 callrate,labeling
0,2020.12.31,0.61,2020.12.01,0.51,up
1,2020.12.30,0.57,2020.11.30,0.60,down
2,2020.12.29,0.54,2020.11.27,0.52,up
3,2020.12.28,0.55,2020.11.26,0.53,up
4,2020.12.24,0.55,2020.11.25,0.51,up
...,...,...,...,...,...
3465,2007.01.09,4.62,2006.12.07,4.36,up
3466,2007.01.08,4.61,2006.12.06,4.49,up
3467,2007.01.05,4.66,2006.12.05,4.46,up
3468,2007.01.04,4.67,2006.12.04,4.47,up


In [16]:
df3.to_csv(r"C:\Users\kyt50\anaconda3\kdigital\project\콜금리데이터프레임2.csv", encoding='utf-8', index=False) #csv파일로 저장